In [ ]:
from nba_api.stats.endpoints import PlayerCareerStats
from nba_api.stats.static import players, teams
import pandas as pd

# dataframe to hold all the relevant information about the players
column_names = ['Player Full Name', 'Player First Name', 'Player Last Name', 'Player API ID', 'Teams']
player_df = pd.DataFrame(columns=column_names)

# gets all active nba players right now 
active_players = players.get_active_players()

# loops through every player in active players and gets relevant information
for player in active_players:
    id = player['id']
    player_stats = PlayerCareerStats(player_id=id).get_data_frames()[0]
    player_teams_id = []

    # get the team ids of the team player has played for
    for team in player_stats['TEAM_ID']:
        if team == 0:
            continue
        # this checks to make sure no duplicate teams are added unless they went back to the team (eg: Lebron)
        if len(player_teams_id) > 0 and team == player_teams_id[-1]:
            continue
        player_teams_id.append(team)

    # gets the names of the team players have played for from ids
    player_team_names = []
    for team in player_teams_id:
        player_team_names.append(teams.find_team_name_by_id(team)['full_name'])

    # other relevant information that will be added to the database
    full_name = player['full_name']
    first_name = player['first_name']
    last_name = player['last_name']
    new_entry = {
        'Player Full Name': full_name,
        'Player First Name': first_name,
        'Player Last Name': last_name,
        'Player API ID': id,
        'Teams': player_team_names
    }
    # adds the player info to the dataframe
    player_df.loc[len(player_df)] = new_entry

# creates a csv file with the player info for future use
player_df.to_csv('final_player_df.csv')

In [80]:
# dataframe created to hold relevant team information
team_columns = ['Team Name', 'Location', 'Nickname', 'Team API ID', 'Image URL']
team_df = pd.DataFrame(columns=team_columns)

# image logo for teams (need to add more which is done in the for loop)
image_url = "https://cdn.nba.com/logos/nba/"

# gets all the teams from the api and loops through them and adds relevant info to the df
all_teams = teams.get_teams()
for team in all_teams:
    team_entry = {
        'Team Name': team['full_name'],
        'Location': team['city'],
        'Nickname': team['nickname'],
        'Team API ID': team['id'],
        'Image URL': image_url + str(team['id']) + "/primary/L/logo.svg"
    }
    team_df.loc[len(team_df)] = team_entry

# writes the dataframe to a csv file for further use
team_df.to_csv('final_team.csv')

In [2]:
# insertion into the player table
import pandas as pd

print("INSERT INTO player(playerID, playerFullName, firstName, lastName, playerApiID) \n VALUES")
player_df = pd.read_csv('final_player_df.csv')
for index, row in player_df.iterrows():
    playerID = index
    playerFullName = row['Player Full Name']
    firstName = row['Player First Name']
    lastName = row['Player Last Name']
    playerApiID = row['Player API ID']

    print(f'({playerID}, "{playerFullName}", "{firstName}", "{lastName}", {playerApiID}),')

INSERT INTO player(playerID, playerFullName, firstName, lastName, playerApiID) 
 VALUES
(0, "Precious Achiuwa", "Precious", "Achiuwa", 1630173),
(1, "Steven Adams", "Steven", "Adams", 203500),
(2, "Bam Adebayo", "Bam", "Adebayo", 1628389),
(3, "Ochai Agbaji", "Ochai", "Agbaji", 1630534),
(4, "Santi Aldama", "Santi", "Aldama", 1630583),
(5, "Nickeil Alexander-Walker", "Nickeil", "Alexander-Walker", 1629638),
(6, "Grayson Allen", "Grayson", "Allen", 1628960),
(7, "Jarrett Allen", "Jarrett", "Allen", 1628386),
(8, "Jose Alvarado", "Jose", "Alvarado", 1630631),
(9, "Kyle Anderson", "Kyle", "Anderson", 203937),
(10, "Giannis Antetokounmpo", "Giannis", "Antetokounmpo", 203507),
(11, "Thanasis Antetokounmpo", "Thanasis", "Antetokounmpo", 203648),
(12, "Cole Anthony", "Cole", "Anthony", 1630175),
(13, "OG Anunoby", "OG", "Anunoby", 1628384),
(14, "Ryan Arcidiacono", "Ryan", "Arcidiacono", 1627853),
(15, "Deni Avdija", "Deni", "Avdija", 1630166),
(16, "Deandre Ayton", "Deandre", "Ayton", 162902

In [ ]:
# insertion into team table
team_df = pd.read_csv('final_team.csv')
print("INSERT INTO team(teamID, teamFullName, location, nickname, teamApiID, imageURL) \n VALUES")
for index, row in team_df.iterrows():
    teamID = index
    teamFullName = row['Team Name']
    location = row['Location']
    nickname = row['Nickname']
    teamApiID = row['Team API ID']
    imageURL = row['Image URL']
    
    print(f'({teamID}, "{teamFullName}", "{location}", "{nickname}", {teamApiID}, "{imageURL}"),')

In [ ]:
# insertion into the playedFor table
import ast

print("INSERT INTO playedFor(playerID, teamID, duplicateID) \n VALUES")
player_df = pd.read_csv('final_player_df.csv')
team_df = pd.read_csv('final_team.csv')
for player_index, row in player_df.iterrows():
    player_teams = ast.literal_eval(row['Teams'])
    count = 0
    for team in player_teams:
        for team_index, row in team_df.iterrows():
            if team == row['Team Name']:
                print(f'({player_index}, {team_index}, {count}),')
                count += 1
                break
